In [1]:
# import glassdoor_scraping as gs
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime

In [4]:
#jobs website

def get_jobs(job_title, location, num_jobs):
    path = "/Users/cristianying/Desktop/glassdoor_salary_proj/chromedriver"
    driver = webdriver.Chrome(path)
    url = 'https://www.glassdoor.com.hk/Job/index.htm'
    driver.get(url)
    # driver.close() = close the tab .quit close window
    #when searching try first: ID, name, class

    search_job = driver.find_element_by_id("KeywordSearch")
    search_location = driver.find_element_by_id("LocationSearch")
    search_job.send_keys(job_title)
    search_location.clear()
    search_location.send_keys(location)
    search_location.send_keys(Keys.RETURN)
    jobs = []
    now = datetime.datetime.now()
    #wait for page to load first before finding the element

    while len(jobs) < num_jobs:
        #wait for page to load
        time.sleep(3)
        try:
            driver.find_element_by_xpath(".//li[contains(@class, 'react-job-listing')]").click()
        except ElementClickInterceptedException:
            pass
        #wait to check if sign in pops up
        time.sleep(5) 

        try:
            driver.find_element_by_xpath(".//span[@alt='Close']").click()  #clicking to the X.
        except NoSuchElementException:
            pass
        try:
            job_list = WebDriverWait(driver, 30).until(
                EC.presence_of_all_elements_located((By.XPATH, ".//li[contains(@class, 'react-job-listing')]"))
            )
            for company in job_list:
                print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
                if len(jobs) >= num_jobs:
                    break
                company.click()  #You might 
                collected_successfully = False

                #these elements exist for all companies, will wait until page loaded and data is retrieved
                while not collected_successfully:
                    try:
                        job_insights = driver.find_element_by_xpath("/html/body/div[3]/div/div/div[1]/div/div[2]/section/div/div/article/div/div[1]/div/div/div[3]/div[2]")

                        company_name = driver.find_element_by_xpath(".//div[@class = 'css-xuk5ye e1tk4kwz5']").text
                        job_type = job_insights.find_element_by_xpath(".//div[1]").text
                        industry = job_insights.find_element_by_xpath(".//div[3]").text
                        company_size = job_insights.find_element_by_xpath(".//div[4]").text
                        job_description = driver.find_element_by_xpath(".//div[@class='jobDescriptionContent desc']").text                    
                        job_title = driver.find_element_by_xpath(".//div[@class='css-1j389vi e1tk4kwz2']").text                    
                        collected_successfully = True
                    except:
                        #wait until company div shows up
                        time.sleep(1)

                #below are separated from the while loop, in some cases they do not exist
                try:
                    founded = driver.find_element_by_xpath(".//span[contains(text(), 'Founded')]/following-sibling::span").get_attribute('innerHTML')
                except NoSuchElementException: 
                    founded = now.year
                try:
                    company_type = driver.find_element_by_xpath(".//span[contains(text(), 'Type') and @class = 'css-1pldt9b e1pvx6aw1']/following-sibling::span").get_attribute('innerHTML')
                except NoSuchElementException: 
                    company_type = 'NA'
                try:
                    sector = driver.find_element_by_xpath(".//span[contains(text(), 'Sector') and @class = 'css-1pldt9b e1pvx6aw1']/following-sibling::span").get_attribute('innerHTML')
                except NoSuchElementException: 
                    sector = 'NA'
                try:
                    rating = driver.find_element_by_xpath(".//div[@class = 'mr-sm css-ey2fjr e1pr2f4f3']").text
                except NoSuchElementException: 
                    rating = -1
                try:
                    base_salary = company.find_element_by_xpath(".//span[@data-test='detailSalary']").text                    
                except NoSuchElementException:
                    base_salary = -1 #You need to set a "not found value. It's important."

                #add job details to array
                jobs.append(
                    {
                        "company_name" : company_name,
                        "job_type" : job_type,
                        "base_salary" : base_salary,
                        "industry" : industry,
                        "company_size" : company_size,
                        "founded" : founded,
                        "rating" : rating,
                        "company_type" : company_type,
                        "sector" : sector,
                        "job_description" : job_description,
                        "job_title" : job_title
                    }
                )
            try:
                driver.find_element_by_xpath(".//a[@data-test='pagination-next']").click()
            except NoSuchElementException: 
                print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(30, len(jobs)))
                break
        
        except BaseException as err:
            print(f"Unexpected {err=}, {type(err)=}")
        

    #     time.sleep(5)

    #     try:
    #         driver.find_element_by_xpath(".//span[@alt='Close']").click()
    #     except NoSuchElementException: 
    #         print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(30, len(jobs)))
    #         break

    return pd.DataFrame(jobs)

In [7]:
df_jobs = get_jobs("data analyst", "Hong Kong", 1000)
df_jobs

Progress: 0/1000
Progress: 1/1000
Progress: 2/1000
Progress: 3/1000
Progress: 4/1000
Progress: 5/1000
Progress: 6/1000
Progress: 7/1000
Progress: 8/1000
Progress: 9/1000
Progress: 10/1000
Progress: 11/1000
Progress: 12/1000
Progress: 13/1000
Progress: 14/1000
Progress: 15/1000
Progress: 16/1000
Progress: 17/1000
Progress: 18/1000
Progress: 19/1000
Progress: 20/1000
Progress: 21/1000
Progress: 22/1000
Progress: 23/1000
Progress: 24/1000
Progress: 25/1000
Progress: 26/1000
Progress: 27/1000
Progress: 28/1000
Progress: 29/1000
Progress: 30/1000
Progress: 31/1000
Progress: 32/1000
Progress: 33/1000
Progress: 34/1000
Progress: 35/1000
Progress: 36/1000
Progress: 37/1000
Progress: 38/1000
Progress: 39/1000
Progress: 40/1000
Progress: 41/1000
Progress: 42/1000
Progress: 43/1000
Progress: 44/1000
Progress: 45/1000
Progress: 46/1000
Progress: 47/1000
Progress: 48/1000
Progress: 49/1000
Progress: 50/1000
Progress: 51/1000
Progress: 52/1000
Progress: 53/1000
Progress: 54/1000
Progress: 55/1000
Pr

Progress: 437/1000
Progress: 438/1000
Progress: 439/1000
Progress: 440/1000
Progress: 441/1000
Progress: 442/1000
Progress: 443/1000
Progress: 444/1000
Progress: 445/1000
Progress: 446/1000
Progress: 447/1000
Progress: 448/1000
Progress: 449/1000
Progress: 450/1000
Progress: 451/1000
Progress: 452/1000
Progress: 453/1000
Progress: 454/1000
Progress: 455/1000
Progress: 456/1000
Progress: 457/1000
Progress: 458/1000
Progress: 459/1000
Progress: 460/1000
Progress: 461/1000
Progress: 462/1000
Progress: 463/1000
Progress: 464/1000
Progress: 465/1000
Progress: 466/1000
Progress: 467/1000
Progress: 468/1000
Progress: 469/1000
Progress: 470/1000
Progress: 471/1000
Progress: 472/1000
Progress: 473/1000
Progress: 474/1000
Progress: 475/1000
Progress: 476/1000
Progress: 477/1000
Progress: 478/1000
Progress: 479/1000
Progress: 480/1000
Progress: 481/1000
Progress: 482/1000
Progress: 483/1000
Progress: 484/1000
Progress: 485/1000
Progress: 486/1000
Progress: 487/1000
Progress: 488/1000
Progress: 48

Progress: 869/1000
Progress: 870/1000
Progress: 871/1000
Progress: 872/1000
Progress: 873/1000
Progress: 874/1000
Progress: 875/1000
Progress: 876/1000
Progress: 877/1000
Progress: 878/1000
Progress: 879/1000
Progress: 880/1000
Progress: 881/1000
Progress: 882/1000
Progress: 883/1000
Progress: 884/1000
Progress: 885/1000
Progress: 886/1000
Progress: 887/1000
Progress: 888/1000
Progress: 889/1000
Progress: 890/1000
Progress: 891/1000
Progress: 892/1000
Progress: 893/1000
Progress: 894/1000
Progress: 895/1000
Progress: 896/1000
Progress: 897/1000
Progress: 898/1000
Progress: 899/1000
Progress: 900/1000
Progress: 901/1000
Progress: 902/1000
Progress: 903/1000
Progress: 904/1000
Progress: 905/1000
Progress: 906/1000
Progress: 907/1000
Progress: 908/1000
Progress: 909/1000
Unexpected err=StaleElementReferenceException('stale element reference: element is not attached to the page document\n  (Session info: chrome=94.0.4606.81)', None, None), type(err)=<class 'selenium.common.exceptions.Stale

,company_name,job_type,base_salary,industry,company_size,founded,rating,company_type,sector,job_description,job_title
0,"ExodusPoint Capital Management Hong Kong, Limi...",Job Type: Full-time,HK$210K - HK$368K (Glassdoor Est.),Industry: Finance,Size: 501 to 1000 Employees,2017,3.2,Company - Private,Finance,"ExodusPoint Capital, founded in 2017 by Michae...",Research analyst - Quantitative Strategies
1,MatchTalent Limited\n4.0,Job Type: Full-time,-1,Industry: N/A,Size: Unknown,2021,4.0,Company - Private,NA,Responsibilities:\nPerform scheduling post-tra...,Post Trade Data Analyst
2,Sun Life Hong Kong Ltd\n4.5,Job Type: Full-time,-1,Industry: N/A,Size: Unknown,2021,4.5,Company - Private,NA,About Sun Life Financial\n\nSun Life Financial...,"Data Analyst, Business Intelligence & Solutions"
3,The Hong Kong Jockey Club\n3.4,Job Type: Part-time,-1,"Industry: Arts, Entertainment & Recreation",Size: 10000+ Employees,1884,3.4,Non-profit Organisation,"Arts, Entertainment &amp; Recreation",The Department\nThe Customer Digital Experienc...,Part-time Data Analyst Assistant
4,5 Miles Lab Company Limited,Job Type: Full-time,HK$180K - HK$336K (Employer Est.),Industry: N/A,Size: N/A,2021,-1,NA,NA,Job Scope\nDevelopment of websites and applica...,Data Analyst @ Agency
...,...,...,...,...,...,...,...,...,...,...,...
995,Wings Recruitment Partners Limited,Job Type: N/A,-1,Industry: N/A,Size: 1 to 50 Employees,2021,-1,Company - Private,NA,Wings Recruitment Partners is a boutique recru...,Data Analyst
996,Bloomberg\n4.1,Job Type: N/A,-1,Industry: Information Technology,Size: 10000+ Employees,1981,4.1,Company - Private,Information Technology,"Bloomberg runs on data, and data drives the ma...",2022 Market Data Analyst (Chinese Speaker) - H...
997,Cigna\n3.7,Job Type: Full-time,HK$192K - HK$360K (Glassdoor Est.),Industry: Insurance,Size: 10000+ Employees,1865,3.7,Company - Public,Insurance,Job Objective:\nTo sustain company’s growth bu...,"Financial Data Analyst, International Markets"
998,China Life Insurance Company Limited\n2.9,Job Type: N/A,-1,Industry: Insurance,Size: 10000+ Employees,2021,2.9,Company - Public,Insurance,China Life Insurance (Overseas) Company Limite...,Data Analyst


In [11]:
df_jobs
df_jobs.to_csv('glassdoor_jobs.csv', index = False)